### using F(2x2,3x3) convolution with m=2, r=3, with input tile size(m+n-1)x(m+n-1) i.e 4x4,
###  total image_size= 32x32x3;
### input_tile = 4x4;
### winconv2d output= 30x30x1;
### stride 2 for 1 filter of size 3x3;

In [ ]:
from keras.preprocessing.image import load_img, img_to_array
import matplotlib.pyplot as plt
ab= load_img('C:\drive\IMG\center_2018_06_01_11_38_42_499.jpg', target_size= (66,66,3))
abarr= np.array(ab)
print(abarr.shape)
plt.imshow(ab)

In [ ]:
ab= create_tiles(abarr,(4,4),(2,2))
print(ab.shape)
plt.imshow(ab[18,18])

# Winograd convolution block

#### Create _tiles function for obtaining input image tiles of shape (4,4) at stride (2,2)

In [ ]:
def create_tiles(image, tsize, st):
    '''create input tiles of shape (m+n1)x(m+n-1) i.e (4, 4) in spatial plane'''
    
    ishape = np.array(image.shape[:2])#takes spatial dimensions into account (66,66)
    tshape = np.array(tsize)
    stride = np.array(st)
    tcount_per_layer = tuple((ishape - tshape)//stride + 1) #gives 32x32 array with each element corresponding to 4x4 tile
    # means 32x32 are to be divided into 225, 4x4 shaped tiles across each channel
    
    #image has 3 channels with 66x66 elements in each
    
    #strided_arr= np.empty(tcount_per_layer+ tsize + image.shape[-1:], dtype= np.uint8) #yields shape (32,32,4,4,3)    
    strided_arr= np.empty((tcount_per_layer+ image.shape[-1:]+ tsize), dtype= np.uint8)#yields shape (32,32,3,4,4)##changes made'''
    
    #image is of (66,66,3) dimension
    for idx1 in range(strided_arr.shape[0]): #stores(4,4,3) slices in shape of (3,4,4) in (15,15) spatial postions
        for idx2 in range(strided_arr.shape[1]):
            strided_arr[idx1,idx2] = image[2*idx1:2*idx1+4, 2*idx2:2*idx2+4].reshape(3,4,4)##changes made
            '''storing image array after reshaping from (4,4,3) to (3,4,4)
            
            strided_arr: array of 4x4 input tiles is shaped (32,32,3,4,4)  '''
    
    return strided_arr
    

## obtaining filter transforms for k filters:
* U = (G x g)x(G.T)
* where G is a matrix shaped (4,3), from polynmial interpolation between data(4,4) and filter(3,3) polynomial expressions
* g= 2D matrix of a filter channel, shaped (3,3)
* G.T= transpose of G

In [ ]:
from keras import backend as k

def kernel(num_filters, shape):# argument in tuple form (3,3,3)
    from keras import backend as k
    from keras.initializers import glorot_uniform
    filter_array = k.eval(glorot_uniform()((num_filters,)+shape)) #adding one more dimension to filter_array
    return filter_array

def kernel_tran(num_filters, filter_shape):
    img_filter= kernel(num_filters, filter_shape)#filter_shape (num_filter, 3,3,3)
    G= np.array([[1, 0, 0], [0.5, 0.5, 0.5], [0.5, -0.5, 0.5], [0, 0, 1]])
    k_trans= np.empty((num_filters,filter_shape[0],G.shape[0],G.shape[0]))
    # k_trans array shaped (num_filter, 3,4,4) to hold filter transforms,"channel first" ##changes made
    
    #np.set_printoptions(suppress=True)
    #ft_map = map(lambda x: np.dot(np.dot(G, ker[x]),G.T), range(ker.shape[0]))#alternate to for loop below
    #ft_list =list(ft_map)
    #k_trans2 = np.array(ab1, dtype=np.float16)    
    for i in range(num_filters):
        for j in range(len(filter_shape)):
            k_trans[i,j] = np.dot(np.dot(G,img_filter[i,j]), G.T) #k_trans[i] stores value of (Gxg)x(G.T) for all 3 channels of kernel
            '''outputs kernel transform U= (Gxg)x(G.T) for each of 3 channels of kernel shaped 3x3 spatially.
           k_trans array shaped: (3,4,4)'''
    return k_trans

## obtaining data transforms for an image:
* V = (Bt x d)x(Bt.T)
* where V: data transform shaped (4,4)
* Bt is a matrix shaped (4,4), from polynmial interpolation between data(4,4) and filter(3,3) polynomial expressions
* d= 2D matrix of an image_tile channel, shaped (4,4)
* Bt.T= transpose of Bt

In [ ]:
def data_tran(input_img, t_size, stride):
    '''
    input args:
    input_img= input image_array, converted from PIL instance. shaped (66,66,3)
    t_size = input tike_size, (m+r-1)x(m+r-1) for F(m,r) Conv, shaped (4,4)
    stride= (2,2) stride across spatial dimensions.
    
    output:
    d_trans= data transform, shaped (32,32,4,4,3) obtained as V= (Bxd)x(B.T) for each of 3 channels of each of 32x32
    input image tiles of each shaped 4x4.
    '''
    img_tile_array= create_tiles(input_img, t_size, stride)#yields tile_array of shape (32,32,3,4,4) for input image of (66,66,3)
    Bt= np.array([[1, 0, -1, 0],[0, 1, 1, 0], [0, -1, 1, 0], [0, 1, 0, -1]]) #data transform shaped (4,4)
        
    d_trans= np.empty(img_tile_array.shape, dtype= np.uint8)# array to hold data transform shaped (32,32,3,4,4) ##changes made

    for i in range(d_trans.shape[0]): #range(32)
        for j in range(d_trans.shape[1]): #range(32)
            for k in range(d_trans.shape[2]): #range(3) ##changes made to .shape[4] i.e 3 earlier
                d_trans[i,j,k] = np.dot(np.dot(Bt, img_tile_array[i,j,k]), Bt.T)# #changes made to channel placement
                '''outputs data transform V= (Bxd)x(B.T) for each of 3 channels of each of 32x32 input image tiles of each
                shaped 4x4.
                d_trans array shaped: (32,32,3,4,4)'''
                
    return d_trans
    
    


## obtain winograd output transforms:
* Y = [At x Summation_across_channels(U.V)] x [At.T]
* where Y: output transform shaped (2,2)
* At is a matrix shaped (2,4), from polynmial interpolation between data(4,4) and filter(3,3) polynomial expressions
* Summation_across_channels(U.V) = a 2D matrix of shape (4,4) obtained as a result of pointwise multiplication between
  corresponding channel of U and V, and a summation over all 3 channels following that.
* At.T= transpose of At

In [ ]:
    
def winoutput(in_array, num_filters, filter_shape, stride):
    '''
    in_array: input array, for first layer, its (32,32,3) array of each image
    filter_shape: shape of filter, ex- (3,3,3)
    stride: spatial stride, ex- (2,2)
    
    ouput:
    out_trans: ouput of F(2x2,3x3) convolution, shaped (64,64) for an input of (66,66,3) 
    '''
    
    #for F(2x2,3x3) convolution with 3x3 filter size and 2x2 output tile, with input tile 4x4 obtaining AT, G, D
    At= np.array(([1, 1, 1, 0],[0, 1, -1, 1])) #inverse transform shaped (2,4)
    
    
    in_tile_size = At.shape[0]+ filter_shape[0]- 1 # (m + r -1)= 4
    in_tile_size = tuple((in_tile_size, in_tile_size))
    
    
    ker_transform = kernel_tran(num_filters, filter_shape)
    #filter transform output in shape (num_filters, 3,4,4), as U= (Gxg)x(G.T) for each of 3 filetr channels, ex- (8,3,4,4)
    
    data_transform= data_tran(in_array, in_tile_size, stride)#shaped (32,32,3,4,4)
    
    out_trans= np.empty((data_transform.shape[0]*2, data_transform.shape[0]*2, num_filters))#output transform shaped
    
    '''(64,64,num_filters) as each 3x4x4 data transform of 32x32 input tiles and 3x4x4 kernel_trans are convolved to form a
    #2x2 matrix, thus 32x32 tiles lead to one spatial feature map shape of (64,64), thus k filters will yield out_trans/feature 
    #map of shape(64,64,k)''' 
    
    for i in range(data_transform.shape[0]):#range(32)
        for j in range(data_transform.shape[1]):#range(32)
            #for k in range(ker_transform.shape[0]):#range(3) ##changes made to use np.sum instead of one more for loop
            for k in range(num_filters):#range(8), for multiple filters used
                
                out_trans[2*i: 2*i+ 2, 2*j: 2*j+ 2, k]= np.dot(np.dot(At, np.sum(ker_transform[k]*data_transform[i,j])),At.T)
                
                '''out_trans[0:2,0:2] mashes kern_transform & data_transform of each channel by pointwise multiplication
                follwed by summation across all 3 channels to give just one (4,4) tile, as:
                At x(summationOver3Channels(U*V))x(At.T) gives a tile shaped (2x2)
                out_trans array shaped: (64,64)
                '''
    return out_trans
    